In [7]:
from sklearn.cluster import KMeans
# common_text에는 파싱된 워드 리스트들이 들어가 있음.
from gensim.test.utils import common_texts
# Doc2Vec이 우리가 텍스트를 사용해서 학습되는 모델(뉴럴넷)이고 
# TaggedDocument가 넘겨주는 텍스트들. 
# 여기서, corpus와 ID들을 함께 넘겨줘야 하는데, 여기서 ID는 tag와 같은 말임
# Q: 여기서, 여러 tag를 함께 넘겨준 다음, 적합한 태그를 찾아주는 방식으로도 처리할 수 있는지 파악하는 것이 필요함. 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# import하기
import json

# Opening JSON file
f = open('./doc_output.json')
awesome_file = json.load(f)

# TaggedDocument는 해당 corpus의 id를 함께 넘겨주는 것을 말합니다. 
# 아래 코드에서 doc에는 단어의 묶음이, tags에는 해당 문서를 표현하는 태그가 들어가게 됨. 
# 흠, 그렇다면, 이 태그에 고유 id가 아닌 다른 것을 넣어주면 현재 문서에 맞는 tag를 찾아주기도 하나? 
# 이렇게 할 경우, words에는 문서를, tags에는 키워드를 넣고, words에서 tags를 예측하는 짓을 할 수도 있지 않을까? 

common_texts_and_tags = [
    (key, value) for key, value in awesome_file.items()
]

#key 관리하기
tags = [i[0] for i in common_texts_and_tags]

print(len(common_texts_and_tags))


5577


In [14]:

# print("##"*20)
# print("tags and its texts")
# print("##"*20)
# for text, tag in common_texts_and_tags:
#     print(f"tag: {tag}, text: {text}")
#     print("\n")

# text: 단어별로 분할되어 있는 리스트 
# tags: 해당 문서를 의미하는 태그. 여기서, tags를 unique id로 넣기는 했는데, 그렇지 않게 넣어서 학습시키는 것도 가능할 것으로 보임. 
TRAIN_documents = [TaggedDocument(words=value, tags=key.split(" ")) for key, value in common_texts_and_tags]

In [15]:
# training. 
# 여러 Parameter들을 사용하여 튜닝할 수 있음. 
model = Doc2Vec(TRAIN_documents, vector_size=100, window=100, epochs=1000, min_count=2, workers=8)
model.build_vocab(TRAIN_documents)

In [5]:
from gensim.test.utils import get_tmpfile
fname = "./my_doc2vec_model3"
model.save(fname)

NameError: name 'model' is not defined

In [8]:
fname = "./repo2vec_model3"
model = Doc2Vec.load(fname)

In [5]:
print(model.total_train_time/3600)

3.267573564666591


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\UC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
def find_key_in_doc_list(doc_list):
  keys = {}
  counts = dict()
  
  f = open('./doc_id_matching.json')
  id_matching = json.load(f)
  # print(f'similar docs list : {doc_list}')
  for (key,value) in doc_list:
    keys[key] = id_matching[key]
  for words in keys.values():
    for word in words.strip().split(" "):
      word = word.replace('-', " ")
      if word in counts:
          counts[word] += 1
      else:
          counts[word] = 1
  for key, value in counts.items():
    if value >= 2 :
      print(key)

In [41]:
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize

test_data = word_tokenize("machine learning algorithm with scikit learn keras this repository contains various machine learning algorithm implemented scikit learn machine learning algorithm such supervised unsupervised simple reinforcement learning sentiment analysis natural language processing supervised simple deep learning algorithm dimensionality reduction bagging boosting implemented scikit learn keras numpy pandas matplotlib tutorials implementation notebook files supervised learning algorithm regression algorithm linear regression multivariate linear regression polynomial regression support vector machine decision trees random forest evaluating regression models using regularization classification algorithm logistic regression nearest neighbour support vector machine kernel support vector machine naive bayes decision trees random forest evaluating classification models unsupervised learning algorithm clustering algorithm means clustering heirarchical clustering association rule learning frequent itemset mining apriori eclat reinforcement learning multi armed bandit upper confidence bound thompson sampling natural language processing simple sentiment analysis using nltk deep learning simple artificial neural network using keras convolutional neural network using keras dimensionality reduction implemented section numpy pandas matplotlib others ipynb principal component analysis linear discriminant analysis kernel pricipal component analysis model selection bagging boosting grid search fold cross validation xgboost")
doc_list = model.dv.most_similar(positive=[model.infer_vector(test_data)],topn=5)
find_key_in_doc_list(doc_list=doc_list)

data visualization
data science
evolutionary algorithm
python


In [ ]:

# document clustering 

print("##"*30)
print("K-Means Clustering")
print("##"*30)
Clustering_Method = KMeans(n_clusters=2, random_state=0)
X = model.docvecs.vectors_docs # document vector 전체를 가져옴. 
Clustering_Method.fit(X)# fitting 
# 결과를 보면 알겠지만, 생각보다 클러스터링이 잘 되지 않음. 
# 일단은 이것 또한 트레이닝 셋이 적어서 그런 것으로 보임. 
cluster_dict = {i:[] for i in range(0, 2)}
for text_tags, label in zip(common_texts_and_tags, Clustering_Method.labels_):
    text, tags = text_tags
    cluster_dict[label].append(text)
for label, lst in cluster_dict.items():
    print(f"Cluster {label}")
    for x in lst:
        print(x)
    print("--"*30)
print("##"*20)

C:\Users\UC\AppData\Local\Temp\ipykernel_1376\1188204071.py:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  X = model.docvecs.vectors_docs # document vector 전체를 가져옴.


############################################################
K-Means Clustering
############################################################


AttributeError: 'KeyedVectors' object has no attribute 'vectors_docs'